In [17]:
import pandas as pd
from torchvision.datasets import ImageFolder
import re

data_path = '/ix/akovashka/arr159/imagenet-r' 
dataset = ImageFolder(root=data_path)

# extract relative path
img_ids = ['/'.join(path.split('/')[-2:]) for path, _ in dataset.samples]

# extract attributes
pattern = re.compile(r"([^/]+)_(\d+)\.jpg$")

attributes=[]
for path, _ in dataset.samples:
    match = pattern.search(path)
    if match:
        transformation_type = match.group(1)
    else:
        raise Exception
    attributes.append(transformation_type)
        
ground_truth_classes = [label for _, label in dataset.samples]
gt_codes = [path.split('/')[-2] for path, _ in dataset.samples]

df = pd.DataFrame({
    'img_id': img_ids,
    'attribute': attributes,
    'gt_code': gt_codes,
    'gt': ground_truth_classes
})

In [7]:
df['attribute'].unique()

array(['art', 'cartoon', 'deviantart', 'embroidery', 'graffiti',
       'graphic', 'misc', 'origami', 'painting', 'sculpture', 'sketch',
       'sticker', 'toy', 'videogame', 'tattoo'], dtype=object)

In [14]:
from os import path

mapping_raw = open(path.join(data_path, 'README.txt')).readlines()
mapping = {line.split()[0]: line.split()[1].rstrip() for line in mapping_raw[13:]}

In [19]:
df.to_csv('mock_data/dataset.csv')

In [22]:
import json
with open('mock_data/mapping.json', 'w') as json_file:
    json.dump(mapping, json_file, indent=4)